从类的继承的角度上来说，子类继承父类的时候，不会自动继承父类的构造函数`__init__`。如果子类想要继承父类的构造函数`__init__`，则需要使用 `super().__init__()`,或者直接引用父类的初始化函数。简单理解，`super().__ init__()`就是用来解决多重继承问题的。

In [4]:
# create a parent class
class Base(object):
    def __init__(self):
        print('Base created')

In [5]:
# create a child class, using parent reference directly.
class ChildA(Base):
    def __init__(self):
        Base.__init__(self)
        print('Child A initialized')

In [6]:
# create other child class, using super with child class
class ChildB(Base):
    def __init__(self):
        print('Child B initialized')
        super(ChildB, self).__init__()

In [7]:
# create third child class, using the super method
class ChildC(Base):
    def __init__(self):
        super().__init__()
        print('Child C initialized')

In [9]:
print('Running ChildA..........')
cA = ChildA()
print('Running ChildB..........')
cB = ChildB()
print('Running ChildC..........')
cC = ChildC()

Running ChildA..........
Base created
Child A initialized
Running ChildB..........
Child B initialized
Base created
Running ChildC..........
Base created
Child C initialized


从运行结果上看，以上三种方式得到的结果是一致的。`super()` 和 `super(ChildB, self).__init__()` 是等同的。

而`Base.__init__(self)` 和 `super().__init__()`，通过下面的code可以看一下`super()`起到的作用。


In [10]:
class Base1:
    def __init__(self):
        print ("Base 1 created")
        super().__init__()
class Base2:
    def __init__(self):
        print ("Base 2 created")
        super().__init__()
class A1(Base1, Base2):
    def __init__(self):
        super().__init__()
        print ("Child A1 initialized")

class A2(Base2, Base1):
    def __init__(self):
        super().__init__()
        print ("Child A2 initialized")


In [11]:
a1 = A1()
print ("\n\n")
a2 = A2()

Base 1 created
Base 2 created
Child A1 initialized



Base 2 created
Base 1 created
Child A2 initialized


在子类中调用父类初始函数时，`super()`会根据子类继承父类的输入顺序进行排序。Python知道哪个模块需要被先引用。

In [25]:
class Base1:
    def __init__(self):
        print ("Base 1 created")
        super().__init__()
class Base2:
    def __init__(self):
        print ("Base 2 created")
        super().__init__()
class A1(Base1, Base2):
    def __init__(self):
        Base1().__init__()
        # Base2().__init__()
        print ("Child A1 initialized")

class A2(Base2, Base1):
    def __init__(self):
        Base2().__init__()
        Base1().__init__()
        print ("Child A2 initialized")


In [26]:
a1 = A1()
print ("\n\n")
a2 = A2()

Base 1 created
Base 1 created
Child A1 initialized



Base 2 created
Base 2 created
Base 1 created
Base 1 created
Child A2 initialized


但是，如果使用`base().__init__()`来初始化，你需要初始化两次`base1().__init__()`，`base2().__init__()`，且存在重复初始化的情况。如果只写`base1().__init__()`，则只会初始化一个。

另外，在Python中可以用`mro`去获取某个模块的调用顺序（子类、父类），Method Resolution Order

In [28]:
print(A1.mro())

[<class '__main__.A1'>, <class '__main__.Base1'>, <class '__main__.Base2'>, <class 'object'>]


In [34]:
class Base1:
    def __init__(self):
        print ("Base 1 created")
        super().__init__()
class Base2:
    def __init__(self):
        print ("Base 2 created")
        # super().__init__()
class A1(Base1, Base2):
    def __init__(self):
        super().__init__()
        print ("Child A1 initialized")

class A2(Base2, Base1):
    def __init__(self):
        super().__init__()
        print ("Child A2 initialized")


In [35]:
a1 = A1()
print ("\n\n")
a2 = A2()

print(A1.mro())

Base 1 created
Base 2 created
Child A1 initialized



Base 2 created
Child A2 initialized
[<class '__main__.A1'>, <class '__main__.Base1'>, <class '__main__.Base2'>, <class 'object'>]


Comment out the super calls in base class and check the output of your script. Having Base1 and Base2 in the mro list, `mro` won’t resolve the order unless the `super()` function is propogated all the way up to the base class i.e., Python propogates the search for the attribute only until it finds one. Comment the init method in Base1 and see for yourself

另外的例子：

In [1]:
class Router:
    def __init__(self, name, mode, number):
        self.name = name
        self.mode = mode
        self.number = number
        self.l3protocol = 'arp, static, rip, eigrp, ospf, isis, bgp'
        
    def desc(self):
        print(f'This is {self.name}_{self.mode}_{self.number}router')
        print(f'l3func: {self.l3protocol}')
        
class Switch(Router):

    def __init__(self, name, mode, number, zone):
        self.zone = zone
        super().__init__(name, mode, number) # 继承super class __init__属性

    def desc(self):
        print(
            f'This is {self.name}_{self.mode}_{self.number} switch. made in {self.zone}')
        print(f'New feature: {self.l3protocol}')
        
if __name__ == '__main__':
    huawei = Switch('HUAWEI', 'CE', '12808', 'USA')
    huawei.l3protocol = 'css, vxlan, evpn, mbgp, srv6'
    huawei.desc()

This is HUAWEI_CE_12808 switch. made in USA
New feature: css, vxlan, evpn, mbgp, srv6


In [3]:
# 在子类没有init的情况下，先运行子类——>父类的init
class Cisco:
    def __init__(self):
        print('cisco')

class Huawei(Cisco):
    print('huawei')

if __name__ == '__main__':
    c = Huawei()

huawei
cisco


In [5]:
# 在子类定义了init的情况下，不再调用父类的init
class Cisco:

    def __init__(self):
        print('cisco')


class Huawei(Cisco):

    def __init__(self):
        print('huawei')

if __name__ == '__main__':
    c = Huawei()

huawei


In [6]:
# 在子类有init，并在子类中使用了super后，会先继承父类中的init方法，在运行子类的内容
class Cisco:

    def __init__(self, name):
        self.name = name
        print(self.name)
        
class Huawei(Cisco):

    def __init__(self, name):
        super().__init__(name)
        print('huawei')

if __name__ == '__main__':
    c = Huawei('cisco')

cisco
huawei
